In [1]:
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from math import exp
from math import log


In [2]:
def parseData(fname):
    for l in urlopen(fname):
        yield eval(l)

print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("done")

def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))


Reading data...
done


In [3]:
def feature(datum):
    feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
    return feat

In [43]:
def feature(datum):
    datum_text=datum['review/text'].lower().split()
    feat = [1, datum_text.count("lactic"),datum_text.count("tart"),datum_text.count("sour"),datum_text.count("citric"),datum_text.count("sweet"),datum_text.count("acid"),datum_text.count("hop"),datum_text.count("fruit"),datum_text.count("salt"),datum_text.count("spicy")]
    return feat




In [23]:
X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]
X_train = X[:int(len(X)/3)]
X_valid = X[int(len(X)/3):int(2*len(X)/3)]
X_test = X[int(2*len(X)/3):]
y_train = y[:int(len(y)/3)]
y_valid = y[int(len(y)/3):int(2*len(y)/3)]
y_test = y[int(2*len(y)/3):]
X[0]

[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [52]:
N=len(y_train)
coeff_1=N/(2*sum(y_train))
coeff_0=N/(2*(N-sum(y_train)))
print(coeff_1)
print(coeff_0)

0.8914206247325631
1.1386990981142389


In [8]:
print('the number in training set is ',len(y_train))
print('num of positive',sum(y_train))
print('number of negative',len(y_train)-sum(y_train))
print('coefficient before loglikehood is ',len(y_train)/(2*sum(y_train)))
print('coefficient before loglikehood for y =0 is ',)

the number in training set is  16666
num of positive 9348
number of negative 7318


In [53]:
##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        if y[i]:
            loglikelihood -= log(1 + exp(-logit))*coeff_1
        else:
            loglikelihood -= (logit+log(1+exp(-logit)))*coeff_0
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0]*len(theta)
    for i in range(len(X)):
        logit = inner(X[i], theta)
        for k in range(len(theta)):
            if y[i]:
                dl[k] += X[i][k] * (1 - sigmoid(logit))*coeff_1
            if not y[i]:
                dl[k] += X[i][k]*(-coeff_0)+X[i][k] * (1 - sigmoid(logit))*coeff_0
    for k in range(len(theta)):
        dl[k] -= lam*2*theta[k]
    return numpy.array([-x for x in dl])

In [54]:
##################################################
# Train                                          #
##################################################
def train(lam):
    theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X_train[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
    return theta


In [64]:
##################################################
# Predict                                        #
##################################################

def performance_valid(theta):
    scores_valid = [inner(theta,x) for x in X_valid]
    predictions_valid = [s > 0 for s in scores_valid]
    correct_valid = [(a==b) for (a,b) in zip(predictions_valid,y_valid)]
    acc_valid = sum(correct_valid) * 1.0 / len(correct_valid)
    return acc_valid
def performance_test(theta):
    scores_test = [inner(theta,x) for x in X_test]
    predictions_test = [s > 0 for s in scores_test]
    correct_test = [(a==b) for (a,b) in zip(predictions_test,y_test)]
    acc_test = sum(correct_test) * 1.0 / len(correct_test)
    return acc_test
def performance_train(theta):
    scores_train = [inner(theta,x) for x in X_train]
    predictions_train = [s > 0 for s in scores_train]
    correct_train = [(a==b) for (a,b) in zip(predictions_train,y_train)]
    acc_train = sum(correct_train) * 1.0 / len(correct_train)
    return acc_train

In [62]:
def evaluate_classifier_test(theta):
    scores_test = [inner(theta,x) for x in X_test]
    predictions_test = [s > 0 for s in scores_test]
    true_positive = [1 if a==1 and b==1 else 0 for (a,b) in zip(predictions_test,y_test)]
    true_negative = [1 if a==0 and b==0 else 0 for (a,b) in zip(predictions_test,y_test)]
    false_positive = [1 if a==1 and b==0 else 0 for (a,b) in zip(predictions_test,y_test)]
    false_negative = [1 if a==0 and b==1 else 0 for (a,b) in zip(predictions_test,y_test)]
    TP=sum(true_positive)
    TN=sum(true_negative)
    FP=sum(false_positive)
    FN=sum(false_negative)
    FPR=FP/(FP+TN)
    FNR=FN/(FN+TP)
    BER=0.5*(FPR+FNR)
    print("number of true positive on test is ",TP)
    print("number of true negative on test is ",TN)
    print("number of false positive on test is ",FP)
    print("number of false negative on test is ",FN)
    print("Balanced Error Rate for test: ",BER)
    # also can calculate the length of ttpp = [ 1 for (a,b) in zip(predictions_test,y_test) if a==1 and b==1]
def evaluate_classifier_train(theta):
    scores_train = [inner(theta,x) for x in X_train]
    predictions_train = [s > 0 for s in scores_train]
    true_positive = [1 if a==1 and b==1 else 0 for (a,b) in zip(predictions_train,y_train)]
    true_negative = [1 if a==0 and b==0 else 0 for (a,b) in zip(predictions_train,y_train)]
    false_positive = [1 if a==1 and b==0 else 0 for (a,b) in zip(predictions_train,y_train)]
    false_negative = [1 if a==0 and b==1 else 0 for (a,b) in zip(predictions_train,y_train)]
    TP=sum(true_positive)
    TN=sum(true_negative)
    FP=sum(false_positive)
    FN=sum(false_negative)
    FPR=FP/(FP+TN)
    FNR=FN/(FN+TP)
    BER=0.5*(FPR+FNR)
    #print("number of true positive on test is ",TP)
    #print("number of true negative on test is ",TN)
    #print("number of false positive on test is ",FP)
    #print("number of false negative on test is ",FN)
    print("Balanced Error Rate for train: ",BER)
def evaluate_classifier_valid(theta):
    scores_valid = [inner(theta,x) for x in X_valid]
    predictions_valid = [s > 0 for s in scores_valid]
    true_positive = [1 if a==1 and b==1 else 0 for (a,b) in zip(predictions_valid,y_valid)]
    true_negative = [1 if a==0 and b==0 else 0 for (a,b) in zip(predictions_valid,y_valid)]
    false_positive = [1 if a==1 and b==0 else 0 for (a,b) in zip(predictions_valid,y_valid)]
    false_negative = [1 if a==0 and b==1 else 0 for (a,b) in zip(predictions_valid,y_valid)]
    TP=sum(true_positive)
    TN=sum(true_negative)
    FP=sum(false_positive)
    FN=sum(false_negative)
    FPR=FP/(FP+TN)
    FNR=FN/(FN+TP)
    BER=0.5*(FPR+FNR)
    #print("number of true positive on test is ",TP)
    #print("number of true negative on test is ",TN)
    #print("number of false positive on test is ",FP)
    #print("number of false negative on test is ",FN)
    print("Balanced Error Rate for valid: ",BER)
    

In [63]:
##################################################
# Validation pipeline                            #
##################################################
lam = 1.0
theta = train(lam)
acc_valid = performance_valid(theta)
acc_test = performance_test(theta)
print("lambda = " + str(lam) + ":\taccuracy for validation set is\t" + str(acc_valid))
print("lambda = " + str(lam) + ":\taccuracy for test set is\t" + str(acc_test))

evaluate_classifier_test(theta)
evaluate_classifier_train(theta)
evaluate_classifier_valid(theta)


lambda = 1.0:	accuracy for validation set is	0.413031739365
lambda = 1.0:	accuracy for test set is	0.59998800024
number of true positive on test is  2378
number of true negative on test is  7622
number of false positive on test is  3133
number of false negative on test is  3534
Balanced Error Rate for test:  0.4445368110876459
Balanced Error Rate for train:  0.4432375407886931
Balanced Error Rate for valid:  0.4199156034378244


In [68]:
#(5)
lam = [0,0.01,0.1]
for i in lam:
    theta = train(i)
    acc_train = performance_train(theta)
    acc_valid = performance_valid(theta)
    acc_test = performance_test(theta)
    print("lambda = " + str(i) + ":\taccuracy for train set is\t" + str(acc_train))
    print("lambda = " + str(i) + ":\taccuracy for validation set is\t" + str(acc_valid))
    print("lambda = " + str(i) + ":\taccuracy for test set is\t" + str(acc_test))


lambda = 0:	accuracy for train set is	0.53726149046
lambda = 0:	accuracy for validation set is	0.412971740565
lambda = 0:	accuracy for test set is	0.59998800024
lambda = 0.01:	accuracy for train set is	0.53726149046
lambda = 0.01:	accuracy for validation set is	0.412971740565
lambda = 0.01:	accuracy for test set is	0.59998800024
lambda = 0.1:	accuracy for train set is	0.53726149046
lambda = 0.1:	accuracy for validation set is	0.412971740565
lambda = 0.1:	accuracy for test set is	0.59998800024


In [67]:
    scores_test = [inner(theta,x) for x in X_test]
    predictions_test = [s > 0 for s in scores_test]
    true_positive = [1 if a==1 and b==1 else 0 for (a,b) in zip(predictions_test,y_test)]
    true_negative = [1 if a==0 and b==0 else 0 for (a,b) in zip(predictions_test,y_test)]
    false_positive = [1 if a==1 and b==0 else 0 for (a,b) in zip(predictions_test,y_test)]
    false_negative = [1 if a==0 and b==1 else 0 for (a,b) in zip(predictions_test,y_test)]
    TP=sum(true_positive)
    TN=sum(true_negative)
    FP=sum(false_positive)
    FN=sum(false_negative)
    FPR=FP/(FP+TN)
    FNR=FN/(FN+TP)
    BER=0.5*(FPR+FNR)
    print("number of true positive on test is ",TP)
    print("number of true negative on test is ",TN)
    print("number of false positive on test is ",FP)
    print("number of false positive on test is ",FN)
    print("Balanced Error Rate: ",BER)
    ttpp = [ 1 for (a,b) in zip(predictions_test,y_test) if a==1 and b==1]
    print(len(ttpp))
    

number of true positive on test is  3920
number of true negative on test is  4940
number of false positive on test is  5815
number of false positive on test is  1992
Balanced Error Rate:  0.4388102836645196
3920
